This file is created to implement the raw Linear Regression model that I created and compare the results with the sklearn implementation of Linear Regression.
I will try my code out on the regression problem of predicting the probability that a student will get an admission in graduate program. 
Lets get Started!!


** The questions I will be answering for myself are as follows: **
* How to handle categorical/nominal/ordinal values in regression problems?
* When do we need to standardize and normalize the data?
* How to know that we are gonna need to add regularization in our model? (I haven't coded regularization part in my code)


In [1]:
import pandas as pd
import numpy as np
import sys
import os
import sklearn.preprocessing

In [4]:
import Linear_Regression as lr

SyntaxError: 'return' outside function (sgd.py, line 61)

In [2]:
df = pd.read_csv('Admission_Predict_Ver1.1.csv')
display(df.head())
display(df.sample())
display(df.info)
display(df.describe())

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
340,341,312,107,3,3.0,3.0,8.46,1,0.75


<bound method DataFrame.info of      Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
0             1        337          118                  4  4.5   4.5  9.65   
1             2        324          107                  4  4.0   4.5  8.87   
2             3        316          104                  3  3.0   3.5  8.00   
3             4        322          110                  3  3.5   2.5  8.67   
4             5        314          103                  2  2.0   3.0  8.21   
5             6        330          115                  5  4.5   3.0  9.34   
6             7        321          109                  3  3.0   4.0  8.20   
7             8        308          101                  2  3.0   4.0  7.90   
8             9        302          102                  1  2.0   1.5  8.00   
9            10        323          108                  3  3.5   3.0  8.60   
10           11        325          106                  3  3.5   4.0  8.40   
11           12     

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000
